# Motion-diffusion Text2Motion Demo

## Setup packages

In [ ]:
import os
import gdown
!git clone https://github.com/GuyTevet/motion-diffusion-model.git

Cloning into 'motion-diffusion-model'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 316 (delta 38), reused 32 (delta 31), pack-reused 252
Receiving objects: 100% (316/316), 2.86 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [ ]:
os.chdir("motion-diffusion-model")
!sudo apt install ffmpeg

# setup python pakage
!pip install --upgrade --no-cache-dir gdown
!gdown "https://drive.google.com/uc?id=1p062yytbpR4U5Lpr6OE-5lf-J5IWWVjy"
!pip install -r requirements.txt

# setup nlp lib
!python3 -m spacy download en_core_web_sm
!pip install git+https://github.com/openai/CLIP.git
!bash prepare/download_glove.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14840 sha256=f2429f1945d3f4ef0c1884a6233d1f413f55434608db7e589480e252eb170ba3
  Stored in directory: /tmp/pip-ephem-wheel-cache-fd4at2wr/wheels/94/8d/0b/bdcd83555c3555f91a33f6c2384428d9f163c7d75ab0d272b4
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Succes

Downloading...
From: https://drive.google.com/uc?id=1p062yytbpR4U5Lpr6OE-5lf-J5IWWVjy
To: /content/motion-diffusion-model/requirements.txt
100% 323/323 [00:00<00:00, 375kB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.2 MB 3.3 MB/s 
     |████████████████████████████████| 50 kB 5.4 MB/s 
     |████████████████████████████████| 96 kB 5.2 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 126 kB 36.4 MB/s 
     |████████████████████████████████| 2.3 MB 36.3 MB/s 
     |████████████████████████████████| 757 kB 38.6 MB/s 
     |████████████████████████████████| 458 kB 43.0 MB/s 
     |████████████████████████████████| 660 kB 51.9 MB/s 
     |████████████████████████████████| 140 kB 49.2 MB/s 
     |████████████████████████████████| 663 kB 56.5 MB/s 
     |████████████████████████████████| 11.1 MB 55.0 MB/s 
     |█████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.3 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.1
    Uninstalling en-core-web-sm-3.4.1:
      Successfully uninstalled en-core-web-sm-3.4.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-eg53e0ix
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-eg53e0ix
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=86d2828df0c47753a0e6c942deadf401847d4a78d61d5f403d8d666c82a4f2ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-1lpm20v6/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully buil

In [ ]:
# download smpl files
!mkdir -p body_models
os.chdir("body_models")

!echo -e "The smpl files will be stored in the 'body_models/smpl/' folder\n"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1INYlGA76ak_cKGzvpOV2Pe6RkYTlXTW2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1INYlGA76ak_cKGzvpOV2Pe6RkYTlXTW2" -O smpl.zip && rm -rf /tmp/cookies.txt
!rm -rf smpl

!unzip smpl.zip
!echo -e "Cleaning\n"
!rm smpl.zip

!echo -e "Downloading done!"
os.chdir("..")

The smpl files will be stored in the 'body_models/smpl/' folder

--2022-11-24 14:35:38--  https://docs.google.com/uc?export=download&confirm=t&id=1INYlGA76ak_cKGzvpOV2Pe6RkYTlXTW2
Resolving docs.google.com (docs.google.com)... 172.217.193.138, 172.217.193.100, 172.217.193.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.193.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-1g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m74e70ueumd0n4ieustoglti7ukd378e/1669300500000/06548236305535658048/*/1INYlGA76ak_cKGzvpOV2Pe6RkYTlXTW2?e=download&uuid=5a3f1bb7-164a-46f0-9e98-b666dcd2146b [following]
--2022-11-24 14:35:38--  https://doc-04-1g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m74e70ueumd0n4ieustoglti7ukd378e/1669300500000/06548236305535658048/*/1INYlGA76ak_cKGzvpOV2Pe6RkYTlXTW2?e=download&uuid=5a3f1bb7-164a-46f0-9e98-b666dcd2146b
Resolving doc-04-1g-docs.googleuserc

In [ ]:
# download t2m evaluators
!echo -e "Downloading T2M evaluators"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1DSaKqWX2HlwBtVH5l7DdW96jeYUIXsOP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1DSaKqWX2HlwBtVH5l7DdW96jeYUIXsOP" -O t2m.zip && rm -rf /tmp/cookies.txt
!rm -rf t2m

!unzip t2m.zip
!echo -e "Cleaning\n"
!rm t2m.zip

!echo -e "Downloading done!"

--2022-11-24 14:35:40--  https://docs.google.com/uc?export=download&confirm=t&id=1DSaKqWX2HlwBtVH5l7DdW96jeYUIXsOP
Resolving docs.google.com (docs.google.com)... 172.217.193.138, 172.217.193.100, 172.217.193.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.193.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-6s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j2gne5gmsopnj3rjtcvvcm6mkej9mni2/1669300500000/11356699582484774286/*/1DSaKqWX2HlwBtVH5l7DdW96jeYUIXsOP?e=download&uuid=622caaa9-9b55-4358-bf7c-3d735afdba9a [following]
--2022-11-24 14:35:40--  https://doc-0k-6s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j2gne5gmsopnj3rjtcvvcm6mkej9mni2/1669300500000/11356699582484774286/*/1DSaKqWX2HlwBtVH5l7DdW96jeYUIXsOP?e=download&uuid=622caaa9-9b55-4358-bf7c-3d735afdba9a
Resolving doc-0k-6s-docs.googleusercontent.com (doc-0k-6s-docs.googleusercontent.com)... 74.125.134.1

### Data

In [ ]:
os.chdir("..")
!git clone https://github.com/EricGuo5513/HumanML3D.git
!unzip ./HumanML3D/HumanML3D/texts.zip -d ./HumanML3D/HumanML3D/
!cp -r HumanML3D/HumanML3D motion-diffusion-model/dataset/HumanML3D

Streaming output truncated to the last 5000 lines.
  inflating: ./HumanML3D/HumanML3D/texts/M009616.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009617.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009618.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009619.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009620.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009621.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009622.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009623.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009624.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009625.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009626.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009627.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009628.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009629.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009630.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009631.txt  
  inflating: ./HumanML3D/HumanML3D/texts/M009632.txt  
  inflating: .

### Pretrain model

In [ ]:
os.chdir("motion-diffusion-model")
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1f8Sd_G53nWxDMvkTnzoouY-8xwOSiW0W" -O model.zip && rm -rf /tmp/cookies.txt
!unzip model.zip -d ./save/

--2022-11-24 14:36:11--  https://docs.google.com/uc?export=download&confirm=&id=1f8Sd_G53nWxDMvkTnzoouY-8xwOSiW0W
Resolving docs.google.com (docs.google.com)... 172.217.204.113, 172.217.204.100, 172.217.204.138, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/roqndl26576472u59gfkrnr9mbkt32f1/1669300500000/01711095037885474985/*/1f8Sd_G53nWxDMvkTnzoouY-8xwOSiW0W?e=download&uuid=b282b15b-3818-4895-8eee-854bfafab6c6 [following]
--2022-11-24 14:36:11--  https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/roqndl26576472u59gfkrnr9mbkt32f1/1669300500000/01711095037885474985/*/1f8Sd_G53nWxDMvkTnzoouY-8xwOSiW0W?e=download&uuid=b282b15b-3818-4895-8eee-854bfafab6c6
Resolving doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)... 74.125.134.13

## Generate

### generate with one line text
- motion_length 動作秒數 2~9.8s
- tetxt_prompt 句子
- num repetitions 生成數量

In [ ]:
num_repetitions = 3
motion_length = 6
text = "a pitcher throwing a ball to catcher"

In [ ]:
!mkdir "../gen"
# generate location npy and mp4
!python3 -m sample.generate --model_path ./save/humanml_trans_enc_512/model000200000.pt \
    --text_prompt "${text}" \
    --motion_length $motion_length \
    --num_repetitions $num_repetitions \
    --output_dir "../gen"

mkdir: cannot create directory ‘../gen’: File exists
Loading dataset...
Reading ././dataset/humanml_opt.txt
Loading dataset t2m ...
100% 4384/4384 [00:00<00:00, 12000.35it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Creating model and diffusion...
TRANS_ENC init
EMBED TEXT
Loading CLIP...
Loading checkpoints from [./save/humanml_trans_enc_512/model000200000.pt]...
### Sampling [repetitions #0]
100% 1000/1000 [00:32<00:00, 30.60it/s]
created 1 samples
### Sampling [repetitions #1]
100% 1000/1000 [00:33<00:00, 29.55it/s]
created 2 samples
### Sampling [repetitions #2]
100% 1000/10

## Display video

In [ ]:
from ipywidgets import Output, GridspecLayout
from IPython import display

os.chdir("..")
grid = GridspecLayout(num_repetitions+2, 1)

i = 0
for df in os.listdir("gen"):
    out = Output()
    if df.endswith(".mp4"):
      with out:
          display.display(display.Video(os.path.join("gen", df), embed=True))
      grid[i, 0] = out
      i += 1
os.chdir("motion-diffusion-model")
grid

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…